In [2]:
# ========================= Imports =========================
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# ========================= Load Dataset =========================
df = pd.read_csv(r"C:\Users\NEVIN\PycharmProjects\DataPreprocessing\Group_ID\data\raw\creditcard.csv")
print("\n=== Dataset Loaded ===")

# ========================= Handle Missing Values =========================
for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

# ========================= Encode Categorical Variables =========================
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# ========================= Outlier Handling =========================
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop('Class', errors='ignore')
for col in num_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = np.clip(df[col], lower, upper)

# ========================= Feature Engineering =========================
if 'Amount' in df.columns and 'Time' in df.columns:
    df['Amount_per_Time'] = df['Amount'] / (df['Time'] + 1)

# ========================= Feature Selection =========================
X_temp = df.drop('Class', axis=1)
y_temp = df['Class']
mi_scores = mutual_info_classif(X_temp, y_temp, random_state=42)
mi_series = pd.Series(mi_scores, index=X_temp.columns).sort_values(ascending=False)
selected_features = mi_series[mi_series > 0].index.tolist()

X = df[selected_features]
y = df['Class']

# ========================= Train/Test Split + Scaling =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ========================= Train only on normal transactions =========================
X_train_normal = X_train_scaled[y_train == 0]
input_dim = X_train_normal.shape[1]

# ========================= Manual Hyperparameter Tuning =========================
param_grid = [
    {'hidden1': 16, 'hidden2': 8, 'lr': 0.001, 'batch_size': 256},
    {'hidden1': 32, 'hidden2': 16, 'lr': 0.001, 'batch_size': 256},
    {'hidden1': 16, 'hidden2': 8, 'lr': 0.005, 'batch_size': 128},
]

best_threshold = None
best_roc = 0
best_model = None

for params in param_grid:
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(params['hidden1'], activation='relu')(input_layer)
    encoded = Dense(params['hidden2'], activation='relu')(encoded)
    decoded = Dense(params['hidden1'], activation='relu')(encoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=params['lr']), loss='mse')

    autoencoder.fit(
        X_train_normal, X_train_normal,
        epochs=20,
        batch_size=params['batch_size'],
        shuffle=True,
        validation_split=0.1,
        verbose=0
    )

    # Compute reconstruction error on test
    X_test_pred = autoencoder.predict(X_test_scaled)
    mse = np.mean(np.power(X_test_scaled - X_test_pred, 2), axis=1)

    # Threshold from normal training data
    X_train_pred = autoencoder.predict(X_train_normal)
    mse_train = np.mean(np.power(X_train_normal - X_train_pred, 2), axis=1)
    threshold = mse_train.mean() + 3 * mse_train.std()

    # Predicted labels
    y_pred = (mse > threshold).astype(int)

    # Evaluate ROC-AUC
    roc = roc_auc_score(y_test, mse)
    print(f"Params: {params}, ROC-AUC: {roc:.4f}")

    if roc > best_roc:
        best_roc = roc
        best_threshold = threshold
        best_model = autoencoder

# ========================= Final Evaluation =========================
X_test_pred = best_model.predict(X_test_scaled)
mse = np.mean(np.power(X_test_scaled - X_test_pred, 2), axis=1)
y_pred = (mse > best_threshold).astype(int)

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, digits=4))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
roc_auc = roc_auc_score(y_test, mse)
print(f"\nROC-AUC Score: {roc_auc:.4f}")



=== Dataset Loaded ===
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step
Params: {'hidden1': 16, 'hidden2': 8, 'lr': 0.001, 'batch_size': 256}, ROC-AUC: 0.9402
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 670us/step
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 5s 728us/step
Params: {'hidden1': 32, 'hidden2': 16, 'lr': 0.001, 'batch_size': 256}, ROC-AUC: 0.9601
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 771us/step
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
Params: {'hidden1': 16, 'hidden2': 8, 'lr': 0.005, 'batch_size': 128}, ROC-AUC: 0.9477
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.9997    0.9861    0.9928     56864
           1     0.0938    0.8367    0.1687        98

    accuracy                         0.9858     56962
   macro avg     0.5468    0.9114    0.5808     56962
weighted avg     0.9982    0.9858    0.9914     56962


=== Confusion Matrix ===
[[56072   792]
 [   1